In [ ]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [ ]:
nb_classes = 10
batch_size = 128
maxepoches = 250
learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20
def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [ ]:
#Loading and splitting the dataset into train, validation and test


(X_Train, y_Train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
X_Train.shape

(50000, 32, 32, 3)

In [ ]:
# Reshape them to batch_size, width,height,#channels
X_train = X_train.reshape(40000, 32, 32, 3)
X_val = X_val.reshape(10000, 32, 32, 3)
X_test = X_test.reshape(10000, 32, 32, 3)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255
X_test /= 255

In [ ]:
init=RandomNormal(mean=0,stddev=0.02)
input_shape = (32, 32, 3) # Input shape of each image
weight_decay = 0.0005
def build_model():
    # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                     input_shape=input_shape,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256,kernel_regularizer=regularizers.l2(weight_decay), name='dense_1'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(10, name='dense_2'))
    model.add(Activation('softmax'))
    return model
teacher = build_model()

sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)

teacher.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])


In [ ]:
# teacher.fit(X_train,Y_train,batch_size=128,epochs=150,verbose=1,callbacks=[reduce_lr],validation_data=(X_val,Y_val))

In [ ]:
teacher.load_weights("Cifar10_Teacher.h5")

In [ ]:
# Y_test = np_utils.to_categorical(y_test, nb_classes)
loss, acc =teacher.evaluate(X_test, y_test, verbose=1)
loss, acc

313/313 [==============================] - 5s 8ms/step - loss: 0.8247 - accuracy: 0.8996


(0.834343671798706, 0.8992000222206116)

In [ ]:
#Collect the dense vector from the previous layer output and store it in a different model
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)

In [ ]:
#Extracting dense representation from the teacher network
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

In [ ]:
#Splitting the training dense vector among N students(in this case 2)
s1Train=train_dense[:,:128]
s2Train=train_dense[:,128:]


s1Val=val_dense[:,:128]
s2Val=val_dense[:,128:]


In [ ]:
def define_model(name):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3), name=name))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform',name='req'+name))

    model.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
    return model

student1 = define_model('s1')
student1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 64)         

TCN


In [ ]:
# Testing with LR
s1=define_model("s1")
s2=define_model("s2")

opt=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
s1.compile(loss='mse', optimizer=opt)
s2.compile(loss='mse', optimizer=opt)

s1.fit(X_train,s1Train,
          batch_size=256,
          epochs=80,
          verbose=1,
          validation_data=(X_val,s1Val))

s2.fit(X_train,s2Train,
          batch_size=256,
          epochs=60,
          verbose=1,
          validation_data=(X_val,s2Val))

Epoch 1/80
157/157 [==============================] - 3s 16ms/step - loss: 1.7962 - val_loss: 1.6681
Epoch 2/80
157/157 [==============================] - 2s 12ms/step - loss: 1.6461 - val_loss: 1.6434
Epoch 3/80
157/157 [==============================] - 2s 15ms/step - loss: 1.6197 - val_loss: 1.6276
Epoch 4/80
157/157 [==============================] - 2s 14ms/step - loss: 1.6028 - val_loss: 1.6036
Epoch 5/80
157/157 [==============================] - 2s 16ms/step - loss: 1.5849 - val_loss: 1.5745
Epoch 6/80
157/157 [==============================] - 2s 15ms/step - loss: 1.5526 - val_loss: 1.5469
Epoch 7/80
157/157 [==============================] - 2s 16ms/step - loss: 1.5339 - val_loss: 1.5263
Epoch 8/80
157/157 [==============================] - 3s 17ms/step - loss: 1.5194 - val_loss: 1.5074
Epoch 9/80
157/157 [==============================] - 2s 15ms/step - loss: 1.5039 - val_loss: 1.4930
Epoch 10/80
157/157 [==============================] - 3s 16ms/step - loss: 1.4872 - val_lo

157/157 [==============================] - 2s 13ms/step - loss: 1.7805 - val_loss: 1.5798
Epoch 2/60
157/157 [==============================] - 3s 17ms/step - loss: 1.5718 - val_loss: 1.5690
Epoch 3/60
157/157 [==============================] - 2s 16ms/step - loss: 1.5566 - val_loss: 1.5614
Epoch 4/60
157/157 [==============================] - 3s 17ms/step - loss: 1.5492 - val_loss: 1.5553
Epoch 5/60
157/157 [==============================] - 3s 17ms/step - loss: 1.5428 - val_loss: 1.5503
Epoch 6/60
157/157 [==============================] - 3s 16ms/step - loss: 1.5445 - val_loss: 1.5459
Epoch 7/60
157/157 [==============================] - 3s 17ms/step - loss: 1.5421 - val_loss: 1.5421
Epoch 8/60
157/157 [==============================] - 3s 17ms/step - loss: 1.5369 - val_loss: 1.5388
Epoch 9/60
157/157 [==============================] - 3s 18ms/step - loss: 1.5314 - val_loss: 1.5357
Epoch 10/60
157/157 [==============================] - 2s 15ms/step - loss: 1.5250 - val_loss: 1.5330


In [ ]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output

output=tensorflow.keras.layers.concatenate([o1,o2])

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm2=Model([s1.get_layer("s1").input,s2.get_layer("s2").input], output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm2.get_layer('d1').set_weights(my_weights)

In [ ]:
i=0
for l in mm2.layers[:len(mm2.layers)-2]:
    l.trainable=False
#     print(l)

mm2.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0002),
              metrics=['accuracy'])

# Without finetune
batch_size = 256
mm2_history=mm2.fit([X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=50,
          verbose=1,
          validation_data=([X_val,X_val], Y_val))

Epoch 1/50
157/157 [==============================] - 2s 12ms/step - loss: 1.6201 - accuracy: 0.5686 - val_loss: 0.7834 - val_accuracy: 0.7802
Epoch 2/50
157/157 [==============================] - 2s 14ms/step - loss: 1.2023 - accuracy: 0.6631 - val_loss: 0.6932 - val_accuracy: 0.8002
Epoch 3/50
157/157 [==============================] - 2s 12ms/step - loss: 1.0957 - accuracy: 0.6788 - val_loss: 0.6666 - val_accuracy: 0.8051
Epoch 4/50
157/157 [==============================] - 2s 15ms/step - loss: 1.0314 - accuracy: 0.6959 - val_loss: 0.6575 - val_accuracy: 0.8062
Epoch 5/50
157/157 [==============================] - 3s 17ms/step - loss: 1.0121 - accuracy: 0.6998 - val_loss: 0.6527 - val_accuracy: 0.8048
Epoch 6/50
157/157 [==============================] - 2s 12ms/step - loss: 0.9982 - accuracy: 0.7020 - val_loss: 0.6493 - val_accuracy: 0.8051
Epoch 7/50
157/157 [==============================] - 2s 14ms/step - loss: 0.9764 - accuracy: 0.7078 - val_loss: 0.6444 - val_accuracy: 0.8048